# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [3]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [5]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/Users/adeniyiharrison/Documents/github/udacity-data-eng/data_modeling_nosql
['/Users/adeniyiharrison/Documents/github/udacity-data-eng/data_modeling_nosql/event_data/2018-11-15-events.csv', '/Users/adeniyiharrison/Documents/github/udacity-data-eng/data_modeling_nosql/event_data/2018-11-22-events.csv', '/Users/adeniyiharrison/Documents/github/udacity-data-eng/data_modeling_nosql/event_data/2018-11-09-events.csv', '/Users/adeniyiharrison/Documents/github/udacity-data-eng/data_modeling_nosql/event_data/2018-11-18-events.csv', '/Users/adeniyiharrison/Documents/github/udacity-data-eng/data_modeling_nosql/event_data/2018-11-04-events.csv', '/Users/adeniyiharrison/Documents/github/udacity-data-eng/data_modeling_nosql/event_data/2018-11-01-events.csv', '/Users/adeniyiharrison/Documents/github/udacity-data-eng/data_modeling_nosql/event_data/2018-11-27-events.csv', '/Users/adeniyiharrison/Documents/github/udacity-data-eng/data_modeling_nosql/event_data/2018-11-10-events.csv', '/Users/adeniyihar

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [14]:
full_data_rows_list = [] 
    
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))
#print(full_data_rows_list)

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [7]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [13]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [5]:
session.execute(
    """
        CREATE KEYSPACE IF NOT EXISTS udacity
        WITH REPLICATION = {
            'class': 'SimpleStrategy',
            'replication_factor': 1
        }
    """
)

#### Set Keyspace

In [7]:
session = cluster.connect("udacity")

### 1. Return the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [18]:
answer1 = """
    SELECT
        artist_name,
        song_title,
        song_length
    FROM song_length
    WHERE session_id = 338
        AND item_in_session = 4
    """

create1 = """
    CREATE TABLE IF NOT EXISTS song_length (
        session_id int,
        item_in_session int,
        artist_name text,
        song_title text,
        song_length float,
        PRIMARY KEY(session_id, item_in_session)
    )
    """

insert1 = """
    INSERT INTO song_length (
        session_id,
        item_in_session,
        artist_name,
        song_title,
        song_length
        )
    VALUES (
        %s, %s, %s, %s, %s
    )
    """

session.execute(create1)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        session.execute(
            insert1, 
            (
                int(line[8]),
                int(line[3]),
                line[0],
                line[9],
                line[5]
            )
        )                 

In [19]:
rows = session.execute(answer1)

for x in rows:
    print(x)

Row(artist_name='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length='495.3073')


### 2.Return only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [11]:
answer2 = """
    SELECT
        artist_name,
        song_title,
        first_name,
        last_name
    FROM songs_in_session
    WHERE user_id = 10
        AND session_id = 182
    """

create2 = """
    CREATE TABLE IF NOT EXISTS songs_in_session (
        user_id int,
        session_id int,
        first_name text,
        last_name text,
        item_in_session int,
        artist_name text,
        song_title text,
        PRIMARY KEY((user_id, session_id), item_in_session)
    )
    """

insert2 = """
    INSERT INTO songs_in_session (
        user_id,
        session_id,
        first_name,
        last_name,
        item_in_session,
        artist_name,
        song_title
        )
    VALUES (
        %s, %s, %s, %s, %s, %s, %s
    )
    """

session.execute(create2)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        session.execute(
            insert2, (
                int(line[10]),
                int(line[8]),
                line[1],
                line[4],
                int(line[3]),
                line[0],
                line[9]
            )
        )         

In [32]:
rows = session.execute(answer2)

for x in rows:
    print(x)

Row(artist_name='Down To The Bone', song_title="Keep On Keepin' On", first_name='Sylvie', last_name='Cruz')
Row(artist_name='Three Drives', song_title='Greece 2000', first_name='Sylvie', last_name='Cruz')
Row(artist_name='Sebastien Tellier', song_title='Kilometer', first_name='Sylvie', last_name='Cruz')
Row(artist_name='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', first_name='Sylvie', last_name='Cruz')


### 3. Return every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [12]:

answer3 = """
    SELECT
        last_name,
        first_name
    FROM who_listened
    WHERE song_name = 'All Hands Against His Own'
    """

create3 = """
    CREATE TABLE IF NOT EXISTS who_listened (
        user_id int,
        session_id int,
        itemInSession int,
        song_name text,
        last_name text,
        first_name text,
        PRIMARY KEY(song_name, user_id, session_id, itemInSession)
    )
    """

insert3 = """
    INSERT INTO who_listened (
        user_id,
        session_id,
        itemInSession,
        song_name,
        last_name,
        first_name,
        )
    VALUES (
        %s, %s, %s, %s, %s, %s
    )
    """

session.execute(create3)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        session.execute(
            insert3, (
                int(line[10]),
                int(line[8]),
                int(line[3]),
                line[9],
                line[4],
                line[1]
            )
        )         
                    

In [36]:
rows = session.execute(answer3)
for x in rows:
    print(x)

Row(last_name='Johnson', first_name='Sara')
Row(last_name='Levine', first_name='Tegan')
Row(last_name='Lynch', first_name='Jacqueline')


In [37]:
drop_query = """
    DROP TABLE IF EXISTS {table_name}
    """

for x in ["who_listened", "songs_in_session", "song_length"]:
    session.execute(
        drop_query.format(table_name=x)
        )

In [38]:
session.shutdown()
cluster.shutdown()